# Preliminaries

In [ ]:
!pip -q install eeconvert
!pip -q install geemap
!pip -q install eefolium
!pip -q install rarfile
!pip -q install rasterstats
!pip -q install rasterio

In [ ]:
import pandas as pd
import geopandas as gpd
import requests
import os
import tarfile
import zipfile
import rarfile
import ee
import eeconvert as eec
import geemap
import eefolium as emap
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import time
from rasterstats import zonal_stats
from shapely.geometry import mapping
from google.colab import files

In [ ]:
ee.Authenticate()
ee.Initialize(project='daveepipon')

In [ ]:
def extract_file(path, to_directory='.'):
    if path.endswith('.zip'):
        opener, mode = zipfile.ZipFile, 'r'
    elif path.endswith('.tar.gz') or path.endswith('.tgz'):
        opener, mode = tarfile.open, 'r:gz'
    elif path.endswith('.tar.bz2') or path.endswith('.tbz'):
        opener, mode = tarfile.open, 'r:bz2'
    elif path.endswith('.rar'):
        opener, mode = rarfile.RarFile, 'r'
    else:
        raise ValueError(f"Could not extract `%s` as no appropriate extractor is found") % path

    cwd = os.getcwd()
    os.chdir(to_directory)

    try:
        file = opener(path, mode)
        try:
            file.extractall()
        finally:
            file.close()
    finally:
        os.chdir(cwd)

# Samal Island Barangay Admin Boundaries

In [ ]:
phl_adm_url = 'https://data.humdata.org/dataset/caf116df-f984-4deb-85ca-41b349d3f313/resource/12457689-6a86-4474-8032-5ca9464d38a8/download/phl_adm_psa_namria_20231106_shp.zip'
phl_adm_filename = 'phl_adm_psa_namria_20231106_shp'

In [ ]:
phl_adm = requests.get(phl_adm_url)
open(phl_adm_filename + '.zip', 'wb').write(phl_adm.content)

925539837

In [ ]:
path_adm = 'phl_adm'

try:
    os.mkdir(path_adm)
except OSError:
    print("Creation of the directory %s failed" % path_adm)
else:
    print("Successfully created the directory %s " % path_adm)

Creation of the directory phl_adm failed


In [ ]:
zip_rw_data = '../' + phl_adm_filename + '.zip'
extract_file(path=zip_rw_data, to_directory=path_adm)

In [ ]:
gdf_admin4 = gpd.read_file('/content/phl_adm/phl_admbnda_adm4_psa_namria_20231106.shp')

In [ ]:
# Filter to Island Garden City of Samal
gdf_samal = gdf_admin4[gdf_admin4['ADM3_EN'].str.contains('Island Garden City of Samal')][['ADM4_EN','ADM4_PCODE','ADM3_EN','AREA_SQKM','geometry']]

In [ ]:
out_dir = '/content/drive/MyDrive/Samal'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

gdf_samal.to_file('/content/drive/MyDrive/Samal/samal.shp', driver='ESRI Shapefile')

# Preliminaries before we run our GEE Scripts

In [ ]:
region = gdf_samal[['ADM4_PCODE','geometry']]
# We need to simplify `region` as Earth Engine's servers are not accommodating our requests
region = region.to_crs('EPSG:32651') # WGS 84 UTM Zone 51N
region['geometry'] = region['geometry'].simplify(100)
region = region.to_crs('EPSG:4326')

In [ ]:
ee_islands = ee.FeatureCollection(region.__geo_interface__)

# VIIRS Nighttime Lights 2020

In [ ]:
collection = (ee.ImageCollection('NOAA/VIIRS/DNB/ANNUAL_V21')
              .filterBounds(ee_islands)
              .filterDate('2020-01-01', '2020-12-31'))

In [ ]:
def calculate_nightlights(image):
    return image.select('average').rename('nightlights')

In [ ]:
collection_with_nightlights = collection.map(calculate_nightlights)

In [ ]:
mean_nightlights_image = collection_with_nightlights.select('nightlights').mean()

In [ ]:
mean_nightlights_clipped = mean_nightlights_image.clipToCollection(ee_islands)

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), '/content/drive/MyDrive')
out_viirs_stats = os.path.join(out_dir, 'samal_viirs_stats_2020.csv')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
geemap.zonal_statistics(mean_nightlights_clipped, ee_islands, out_viirs_stats, statistics_type='MEAN', scale=1000)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/samal_viirs_stats_2020.csv


# VIIRS Nighttime Lights 2021

In [ ]:
collection = (ee.ImageCollection('NOAA/VIIRS/DNB/ANNUAL_V21')
              .filterBounds(ee_islands)
              .filterDate('2021-01-01', '2021-12-31'))

In [ ]:
def calculate_nightlights(image):
    return image.select('average').rename('nightlights')

In [ ]:
collection_with_nightlights = collection.map(calculate_nightlights)

In [ ]:
mean_nightlights_image = collection_with_nightlights.select('nightlights').mean()

In [ ]:
mean_nightlights_clipped = mean_nightlights_image.clipToCollection(ee_islands)

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), '/content/drive/MyDrive')
out_viirs_stats = os.path.join(out_dir, 'samal_viirs_stats_2021.csv')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
geemap.zonal_statistics(mean_nightlights_clipped, ee_islands, out_viirs_stats, statistics_type='MEAN', scale=1000)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/samal_viirs_stats_2021.csv


# ESA World Cover 2020

In [ ]:
collection = (ee.ImageCollection('ESA/WorldCover/v100')
              .filterBounds(ee_islands)
              .filterDate('2020-01-01', '2020-12-31'))

worldcover = collection.first().select('Map')

In [ ]:
# Function to calculate pixel counts per class for each feature
def calculate_pixel_counts(feature):
    clipped = worldcover.clip(feature.geometry())
    reprojected = clipped.reproject(crs=clipped.projection(), scale=10)
    counts = reprojected.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=feature.geometry(),
        scale=10,
        maxPixels=1e13
    ).get('Map')
    rounded_counts = ee.Dictionary(counts).map(lambda key, value: ee.Number(value).round())
    return feature.set('pixel_counts', rounded_counts)

# Map the function over the ee_islands feature collection
results = ee_islands.map(calculate_pixel_counts)

# Convert the results into a list
results_list = results.getInfo()['features']

# Prepare data for pivoting
data = []
for feature in results_list:
    adm4_pcode = feature['properties']['ADM4_PCODE']
    pixel_counts = feature['properties']['pixel_counts']
    if pixel_counts:
        for class_value, count in pixel_counts.items():
            data.append({'ADM4_PCODE': adm4_pcode, 'Class Value': class_value, 'Pixel Count': count})

# Create a DataFrame for pivoting
df = pd.DataFrame(data)

# Pivot the DataFrame
pivoted_df = df.pivot(index='ADM4_PCODE', columns='Class Value', values='Pixel Count')

# Fill missing values with 0 (if needed) and reset index
pivoted_df = pivoted_df.fillna(0).reset_index()

# Save the pivoted table to a CSV file
out_dir = os.path.join(os.path.expanduser('~'), '/content/drive/MyDrive')
out_csv = os.path.join(out_dir, 'samal_worldcover_pixel_counts_pivoted_2020.csv')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

pivoted_df.to_csv(out_csv, index=False)

print(f"Pivoted pixel counts table saved to {out_csv}")


Pivoted pixel counts table saved to /content/drive/MyDrive/samal_worldcover_pixel_counts_pivoted_2020.csv


#ESA World Cover 2021

In [ ]:
collection = (ee.ImageCollection('ESA/WorldCover/v200')
              .filterBounds(ee_islands)
              .filterDate('2020-12-31', '2021-12-31'))

worldcover = collection.first().select('Map')

In [ ]:
# Function to calculate pixel counts per class for each feature
def calculate_pixel_counts(feature):
    clipped = worldcover.clip(feature.geometry())
    reprojected = clipped.reproject(crs=clipped.projection(), scale=10)
    counts = reprojected.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=feature.geometry(),
        scale=10,
        maxPixels=1e13
    ).get('Map')
    rounded_counts = ee.Dictionary(counts).map(lambda key, value: ee.Number(value).round())
    return feature.set('pixel_counts', rounded_counts)

# Map the function over the ee_islands feature collection
results = ee_islands.map(calculate_pixel_counts)

# Convert the results into a list
results_list = results.getInfo()['features']

# Prepare data for pivoting
data = []
for feature in results_list:
    adm4_pcode = feature['properties']['ADM4_PCODE']
    pixel_counts = feature['properties']['pixel_counts']
    if pixel_counts:
        for class_value, count in pixel_counts.items():
            data.append({'ADM4_PCODE': adm4_pcode, 'Class Value': class_value, 'Pixel Count': count})

# Create a DataFrame for pivoting
df = pd.DataFrame(data)

# Pivot the DataFrame
pivoted_df = df.pivot(index='ADM4_PCODE', columns='Class Value', values='Pixel Count')

# Fill missing values with 0 (if needed) and reset index
pivoted_df = pivoted_df.fillna(0).reset_index()

# Save the pivoted table to a CSV file
out_dir = os.path.join(os.path.expanduser('~'), '/content/drive/MyDrive')
out_csv = os.path.join(out_dir, 'samal_worldcover_pixel_counts_pivoted_2021.csv')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

pivoted_df.to_csv(out_csv, index=False)

print(f"Pivoted pixel counts table saved to {out_csv}")


Pivoted pixel counts table saved to /content/drive/MyDrive/samal_worldcover_pixel_counts_pivoted_2021.csv


# Combine all the tables

In [ ]:
df_viirs_2020 = pd.read_csv('/content/drive/MyDrive/samal_viirs_stats_2020.csv')
df_viirs_2021 = pd.read_csv('/content/drive/MyDrive/samal_viirs_stats_2021.csv')
df_worldcover_2020 = pd.read_csv('/content/drive/MyDrive/samal_worldcover_pixel_counts_pivoted_2020.csv')
df_worldcover_2021 = pd.read_csv('/content/drive/MyDrive/samal_worldcover_pixel_counts_pivoted_2021.csv')

In [ ]:
df_viirs_2020.drop(columns=['system:index'], inplace=True)
df_viirs_2021.drop(columns=['system:index'], inplace=True)

In [ ]:
df_viirs_2020.rename(columns={'mean': 'mean_nightlights_2020'}, inplace=True)
df_viirs_2021.rename(columns={'mean': 'mean_nightlights_2021'}, inplace=True)

In [ ]:
dfs = [gdf_samal, df_viirs_2020, df_viirs_2021, df_worldcover_2020,df_worldcover_2021]  # List of dataframes
result = dfs[0]

for df in dfs[1:]:
    result = pd.merge(result, df, on='ADM4_PCODE', how='outer')  # or other merge type

In [ ]:
result.to_csv('samal_ntl2020_ntl2021_worldcover2020_worldcover2021.csv')